In [ ]:
from requests_html import HTMLSession
import calendar
import json
import os

# read issue list
with open('Issuelist/jfe_issuelist.json','r') as f:
    jf = f.read()
JFE_ISSUELIST = json.loads(jf)

### TEST

In [ ]:
test_vol, test_iss = str(89), str(1)
base_url = 'https://www.sciencedirect.com/journal/journal-of-financial-economics/vol/{}/issue/{}'.format(test_vol, test_iss)
s = HTMLSession()
r = s.get(base_url)

In [ ]:
test_url_list = [i.absolute_links.pop() for i in r.html.find('a.article-content-title')]
test_url_list

In [ ]:
test = s.get(test_url_list[-1])

In [ ]:
# title
test.html.find('span.title-text',first=True).text

In [ ]:
# authors
authors = test.html.find('a.author')

In [ ]:
if authors[2].find('span.text.given-name', first=True) is None:
    print('111')

In [ ]:
authors[0].find('span.text.surname', first=True).text

In [ ]:
authors[0].find('span.text.given-name', first=True).text + ' ' + authors[0].find('span.text.surname', first=True).text

In [ ]:
# string
string = test.html.find('div.text-xs', first=True).text
print(string)

In [ ]:
# volume
[i.strip() for i in string.split(',')][0].strip('Volume| ')

In [ ]:
# issue
[i.strip() for i in string.split(',')][1].strip('Issue| ')

In [ ]:
# date
# [i.strip() for i in string.split(',')][2]

In [ ]:
# abstract
test.html.find('div.Abstracts p',first='True').text

In [ ]:
# link
test.html.find('a.doi',first=True).absolute_links.pop()

In [ ]:
'https://doi.org/10.1016/j.jfineco.2018.11.002'.split('.')[-3]

In [ ]:
'https://doi.org/10.1016/j.jfineco.2018.11.002'.split('.')[-2]

### main

In [ ]:
JFE_URL = 'https://www.sciencedirect.com/journal/journal-of-financial-economics/'
MAX_RETRY = 5


def get_jfe(vol,iss):
    s = HTMLSession()
    
    try:
        r = s.get(JFE_URL+'vol/'+vol+'/issue/'+iss)
    except:
        for c , _ in ernumerate(range(MAX_RETRY+1)):
            print('Retry#{} '.format(c+1))
            r = s.get(JFE_URL+'vol/'+vol+'/issue/'+iss)
            if r.status_code == 200:
                break
    
#     print('[{}] '.format(r.status_code), end='')
    print('Volume \033[32m{}\033[m, Issue \033[32m{}\033[m: Total {} '.format(vol,iss,len(r.html.find('a.article-content-title'))))
    
    # get article url list
    url_list = r.html.find('a.article-content-title')
    url_list = [i.absolute_links.pop() for i in url_list]
    
    # create list
    article = {}
    # isse info write
    article['journal'] = 'JFE'
    ###!!!!!
    vol_iss = r.html.find('.js-vol-issue', first=True).text.split(',')
    article['volume'] = vol_iss[0].strip('Volume| ')
    article['issue'] = vol_iss[1].strip('Issue| ')
    date_page = r.html.find('h3.js-issue-status', first=True).text.split('(')
    print(date_page[1].strip(')| '))
    article['date'] = '{}'.format(date_page[1].strip(')| '))
    article['page'] = '{}'.format(date_page[0].strip(' '))
        
    # article info write
    article['article'] = []
    for i, url in enumerate(url_list):
        print('#{} '.format(i+1), end='')
        # request
        try:
            r = s.get(url)
        except:
            for c , _ in ernumerate(range(MAX_RETRY+1)):
                print('Retry#{} '.format(c+1))
                r = s.get(url)
                if r.status_code == 200:
                    break

        # append new dict
        article['article'].append(dict({"no":str(i+1)}))
        article['article'][i]['title'] = r.html.find('span.title-text',first=True).text
        
        # get date trough link
        link = r.html.find('a.doi',first=True).absolute_links.pop()
        month = link.split('.')[-2]
        year = link.split('.')[-3]
        if year.isdigit() & year.isdigit():
            article['article'][i]['date'] = '{} {}'.format(calendar.month_abbr[int(link.split('.')[-2])], link.split('.')[-3])
        else:
            try:
                article['article'][i]['date'] = '{} {}'.format(date_page[1].strip(')| ').split(' ')[0][:3], date_page[1].strip(')| ').split(' ')[1])
            except:
                article['article'][i]['date'] = date_page[1].strip(')| ')
                
    
        # authors
        authors = r.html.find('a.author')
        a_list = []
        for author in authors:
            gname = author.find('span.text.given-name', first=True)
            gname = '' if gname is None else gname.text
            sname = author.find('span.text.surname', first=True)
            sname = '' if sname is None else sname.text
            a_list.append('{} {}'.format(gname, sname))
        article['article'][i]['author'] = [ name.strip() for name in a_list]

    
        # abstract
        try:
            article['article'][i]['abstract'] = r.html.find('div.Abstracts p',first='True').text
        except: 
            article['article'][i]['abstract'] = ''
        author_list = r.html.find('div.text-xs', first=True).text
        article['article'][i]['link'] = link
        
    # write dot md file
    path_name = 'JSON/JFE/Volume {}/Volume {} - Issue {}.json'.format(vol,vol,iss) 
    os.makedirs(os.path.dirname(path_name), exist_ok=True)
    with open(path_name, 'w') as f:
        # get article items
        f.write(json.dumps(article, indent=4))
    print('DONE!')
            
def jfe_main(vol,iss):
    if os.getcwd()[13:] != 'light-speed-engine':
        print('Not in the right folder')
    else:
        # write json
        get_jfe(vol,iss)

In [ ]:
jfe_main(str(37),str(3))

In [ ]:
%%time
for i in range(37,51):
    vol = i
    for item in JFE_ISSUELIST[str(vol)]:
        jfe_main(str(vol),item)